In [1]:
# load libraries

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re
import pandas as pd

In [2]:
# load the pre-trained model and tokenizator:

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"  # ruGPT2 small model from Sberbank AI
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [7]:
# define function to generate text based on prompts:

def generate_text(prompt, max_length=30):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [8]:
# define input and output data, read the input data

input_file = 'sentence_containing.txt'
output_file = "generated_texts.txt"

with open(input_file, "r", encoding="utf-8") as f:
    prompts = f.readlines()

In [ ]:
# text generation:

with open(output_file, "w", encoding="utf-8") as f:
    for prompt in prompts:
        prompt = prompt.strip()  # removing extra spaces and characters
        if not prompt:
            continue  # omit empty lines
        try:
            generated_text = generate_text(prompt, max_length=30)
            generated_sentence = generated_text[len(prompt):].strip()  # remove input prompt (probably it should be kept..)
            f.write(f"{prompt} {generated_sentence}\n")
        except Exception as e:
            print(f"Ошибка при обработке подсказки: {prompt}")
            print(e)
            continue

In [23]:
# relults formatting and counting:

with open("cleaned_generated_texts.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

prompt_patterns = [
    r'Напишите предложение со словом "([^"]+)".',
    r'Придумайте предложение, в котором будет слово "([^"]+)".',
    r'Составьте предложение, используя слово "([^"]+)".'
]

prompt_texts = []

for line in lines:
    for pattern in prompt_patterns:
        match = re.match(pattern, line)
        if match:
            word = match.group(1)
            generated_text = line.split('. ')[1].strip()
            if word.lower() in generated_text.lower():
                prompt_texts.append((word, generated_text, 1))
            else:
                prompt_texts.append((word, generated_text, 0))
            break

# create DataFrame of 3 columns: "word", "text", "score":
df = pd.DataFrame(prompt_texts, columns=["Слово", "Текст", "Оценка"])

# save data in .csv
df.to_csv("prompt_results.csv", index=False)

# print first and last 10 lines
print(df.head(10))
print(df.tail(10))


         Слово                                              Текст  Оценка
0        возле  Примерная презентация Напишите предложение со ...       1
1   спрашивать  Я сама не знаю чего от меня хотят, но вот чтоб...       1
2    крыжовник  Какой сорт ягод подходит для дачи? Напишите пр...       1
3         осёл  Это должно получиться "какая-то" И тогда это б...       1
4      слышать  Что такое голос в произведениях русских писате...       1
5      рисунок  Напишем его в форме: "Рисунок – это изображени...       1
6      тетрадь  Какой ещё тетерь в доме есть? Напишите просто ...       0
7  происходить  Если хотите показать пример, то составьте пред...       1
8         шлем  Если вы не нашли свое предложение на сайте, то...       0
9         ужин  "А вечером я не ела, Напишите предложение со с...       1
         Слово                                              Текст  Оценка
2994      жаль  Приложите картинку к теме и получите следующий...       1
2995     среда                        